# TensorFlow Assignment: Multilayer Perceptron (MLP) and Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: [Jinwei Wan]


Now that you've run through a simple logistic regression model on MNIST, let's see if we can do better (Hint: we can). For this assignment, you'll build a multilayer perceptron (MLP) and a convolutional neural network (CNN), two popular types of neural networks, and compare their performance. Some potentially useful code:

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
_,nin = mnist.train.images.shape
_,nout = mnist.test.labels.shape
nhid = 500

In [3]:
nin,nhid,nout

(784, 500, 10)

In [4]:
# Helper functions for creating weight variables
def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Tensorflow Function that might also be of interest:
# tf.nn.sigmoid()
# tf.nn.relu()

### Multilayer Perceptron

Build a multilayer perceptron for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> fully connected (500 hidden units) -> nonlinearity (Sigmoid/ReLU) -> fully connected (10 hidden units) -> softmax

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

In [5]:
# Model Inputs
x = tf.placeholder("float", [None, 28*28])### MNIST images enter graph here ###
y_ = tf.placeholder("float", [None, 10])### MNIST labels enter graph here ###

# Define the graph
w_h = weight_variable([nin, nhid]) 
b_h = bias_variable([nhid])
w_o = weight_variable([nhid, nout])
b_o = bias_variable([nout])

def mlp_sigm(x, w_h, b_h, w_o,b_o):
    h1 = tf.nn.sigmoid(tf.matmul(x, w_h) + b_h) # this is a basic mlp, think 2 stacked logistic regressions
    pyx = tf.matmul(h1, w_o) + b_o
    return pyx # note that we dont take the softmax at the end because our cost fn does that for us

def mlp_relu(x, w_h, b_h, w_o,b_o):
    h1 = tf.nn.relu(tf.matmul(x, w_h) + b_h) # this is a basic mlp, think 2 stacked logistic regressions
    pyx = tf.matmul(h1, w_o) + b_o
    return pyx # note that we dont take the softmax at the end because our cost fn does that for us

### Create your MLP here##
### Make sure to name your MLP output as y_mlp ###
y_mlp = mlp_sigm(x, w_h,b_h, w_o,b_o)
#y_mlp = mlp_relu(x, w_h,b_h, w_o,b_o)

# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(4000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(100)
                validation_accuracy += (1/10.) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.092
step 250, validation accuracy 0.893
step 500, validation accuracy 0.93
step 750, validation accuracy 0.91
step 1000, validation accuracy 0.925
step 1250, validation accuracy 0.927
step 1500, validation accuracy 0.957
step 1750, validation accuracy 0.953
step 2000, validation accuracy 0.956
step 2250, validation accuracy 0.938
step 2500, validation accuracy 0.957
step 2750, validation accuracy 0.963
step 3000, validation accuracy 0.965
step 3250, validation accuracy 0.953
step 3500, validation accuracy 0.967
step 3750, validation accuracy 0.962
test accuracy 0.9634


In [6]:
y_mlp = mlp_relu(x, w_h,b_h, w_o,b_o)

# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(4000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(100)
                validation_accuracy += (1/10.) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.09
step 250, validation accuracy 0.95
step 500, validation accuracy 0.966
step 750, validation accuracy 0.957
step 1000, validation accuracy 0.97
step 1250, validation accuracy 0.964
step 1500, validation accuracy 0.949
step 1750, validation accuracy 0.977
step 2000, validation accuracy 0.979
step 2250, validation accuracy 0.974
step 2500, validation accuracy 0.975
step 2750, validation accuracy 0.983
step 3000, validation accuracy 0.983
step 3250, validation accuracy 0.974
step 3500, validation accuracy 0.98
step 3750, validation accuracy 0.968
test accuracy 0.9809


#### Comparison

How do the sigmoid and rectified linear unit (ReLU) compare?

***
Answer:

    Test accuracy of sigmoid activation is 96.34%
    Test accuracy of ReLU activation is 98.09%
    Training Time: sigmoid ≈ Relu
    Obviously, Networks here using ReLU activation gets a better classification result than using sigmoid activation.
***

### Convolutional Neural Network

Build a simple 2-layer CNN for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> CNN (32 5x5 filters) -> nonlinearity (ReLU) ->  (2x2 max pool) -> CNN (64 5x5 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> fully connected (1024 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

Some additional functions that you might find helpful:

In [7]:
# Convolutional neural network functions
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Tensorflow Function that might also be of interest:
# tf.reshape()

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

*Hint: Convolutional Neural Networks are spatial models. You'll want to transform the flattened MNIST vectors into images for the CNN. Similarly, you might want to flatten it again sometime before you do a softmax. You also might want to look into the  [conv2d() documentation](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d) to see what shape kernel/filter it's expecting.*

In [8]:
# Model Inputs
x = tf.placeholder("float", [None, 28*28])### MNIST images enter graph here ###
y_ = tf.placeholder("float", [None, 10])### MNIST labels enter graph here ###

# Define the graph
w_conv1 = weight_variable([5,5,1,32]) 
#b_conv1 = bias_variable([32])
w_conv2 = weight_variable([5,5,32,64]) 
#b_conv2 = bias_variable([64])
w_fc3 = weight_variable([7*7*64,1024]) 
b_fc3 = bias_variable([1024])
w_o = weight_variable([1024,10]) 
b_o = bias_variable([10])
def conv_homework(x,w_conv1,w_conv2,w_fc3,b_fc3,w_o,b_o):
    x_image = tf.reshape(x, [-1,28,28,1])# 28*28*1
    h1_conv = conv2d(x_image, w_conv1) # 28*28*32
    h1_pool = max_pool_2x2(h1_conv)# 14*14*32
    h2_conv = conv2d(h1_pool, w_conv2)# 14*14*64
    h2_pool = max_pool_2x2(h2_conv)# 7*7*64
    h2_pool_flat = tf.reshape(h2_pool, [-1, 7*7*64])# reshape
    h3_fc = tf.nn.relu(tf.matmul(h2_pool_flat, w_fc3) + b_fc3) # 
    pyx = tf.matmul(h3_fc, w_o) + b_o
    return pyx # note that we dont take the softmax at the end because our cost fn does that for us

### Create your CNN here##
### Make sure to name your CNN output as y_conv ###

y_conv = conv_homework(x,w_conv1,w_conv2,w_fc3,b_fc3,w_o,b_o)

# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(10000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(50)
                validation_accuracy += (1/10.) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.102
step 250, validation accuracy 0.912
step 500, validation accuracy 0.964
step 750, validation accuracy 0.964
step 1000, validation accuracy 0.958
step 1250, validation accuracy 0.966
step 1500, validation accuracy 0.974
step 1750, validation accuracy 0.98
step 2000, validation accuracy 0.976
step 2250, validation accuracy 0.992
step 2500, validation accuracy 0.974
step 2750, validation accuracy 0.978
step 3000, validation accuracy 0.972
step 3250, validation accuracy 0.984
step 3500, validation accuracy 0.982
step 3750, validation accuracy 0.984
step 4000, validation accuracy 0.982
step 4250, validation accuracy 0.982
step 4500, validation accuracy 0.988
step 4750, validation accuracy 0.986
step 5000, validation accuracy 0.986
step 5250, validation accuracy 0.986
step 5500, validation accuracy 0.988
step 5750, validation accuracy 0.994
step 6000, validation accuracy 0.98
step 6250, validation accuracy 0.99
step 6500, validation accuracy 0.998
step 6750,

Some differences from the logistic regression model to note:

- The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

- The logistic regression model we used previously was pretty basic, and as such, we were able to get away with using the GradientDescentOptimizer, which performs implements the gradient descent algorithm. For more difficult optimization spaces (such as the ones deep networks pose), we might want to use more sophisticated algorithms. Prof David Carlson has a lecture on this later.
    
- Because of the larger size of our network, notice that our minibatch size has shrunk.
    
- We've added a validation step every 250 minibatches. This let's us see how our model is doing during the training process, rather than sit around twiddling our thumbs and hoping for the best when training finishes. This becomes especially significant as training regimens start approaching days and weeks in length. Normally, we validate on the entire validation set, but for the sake of time we'll just stick to 10 validation minibatches (500 images) for this homework assignment.

#### Comparison

How do the MLP and CNN compare in accuracy? Training time? Why would you use one vs the other? Is there a problem you see with MLPs when applied to other image datasets?

***
Problem 1:

    Test accuracy of ReLU activation(MLP) is 98.09% 
    Test accuracy of CNN is 98.78% 
    Acccuracy: CNN > MLP
    Training Time: CNN >> MLP 
    
Problem 2: 

    When your classification problem is easy(e.g. dataset like MNIST), using MLPs is good enough. MLPs have simple structure, training MLPs are very fast and using less calculation resource. 
    But, when your problem is complex(e.g. classify Image data), using CNNs maybe better, thought they are slow in training, but they always get a better result than MLPs.

Problem 3: 

    When using MLPs, each sample(image) must reshape to a vector, but in CNNs the samples are using directly(Matrix format). That means in MLPs, the image structure are destoryed, this makes MLPs get lower accuracy than CNNs when in classification and other tasks.
    
***